# House Prices Prediction Model Training, Evaluation & Selection Notebook

# Part 1 - DEFINE
***

### ---- 1 Define the problem ----

In this notebook, using the Kaggle [dataset](https://www.kaggle.com/c/house-prices-advanced-regression-techniques/data?select=train.csv), first, I'll establish simple baseline model using the OLS regerssion, and then I'll develop a few predictive models, namely, random forest, xgboost and lightgbm and compare the performance of these models against the baseline 

### Objective: 
- To understand what factors contribute most to house prices.

- To create a model that predicts the price of a house with a given several features. 

- To create or improve predictive accuracy of the model compared to baseline model

The implementation of this model will allow housing agencies (e.g., CMHC), real-estate companies, banks, municipial governments and home buyers to make informed decisions.

### Check versions of the Python and some key packages to ensure most recent version is used

In [ ]:
%load_ext watermark
%watermark -a 'Vusal Babashov' -u -d -v -p numpy,mlxtend,matplotlib,sklearn

In [ ]:
#conda update scikit-learn numpy matplotlib 

### Import Libraries

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline


from sklearn.impute import SimpleImputer

from sklearn.pipeline import Pipeline
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

from sklearn.decomposition import PCA

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_validate

from sklearn.compose import ColumnTransformer

from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score, explained_variance_score 
#from sklearn.metrics import mean_absolute_percentage_error

from sklearn.linear_model import LinearRegression
from sklearn.experimental import enable_hist_gradient_boosting
from sklearn.ensemble import RandomForestRegressor, HistGradientBoostingRegressor
import lightgbm as lgb 
import xgboost as xgb

from xgboost import XGBRegressor
from lightgbm import LGBMRegressor


#your info here
__author__ = "Vusal Babashov"
__email__ = "vbabashov@gmail.com"
__website__ = 'https://vbabashov.github.io'

### Load the Data

In [3]:
#load the data into a Pandas dataframe
file_path = "/Users/vusalbabashov/Desktop/house-prices/data/"
df_train = pd.read_csv(file_path + "train.csv")
df_test_feature = pd.read_csv(file_path + "test.csv")
df_test_target = pd.read_csv(file_path + "sample_submission.csv")

## Part 2 - DISCOVER
***

Steps including Obtain data, Clean data, Explore data, and full EDA are implemented in a seperate [notebook](https://github.com/vbabashov/house-prices/blob/main/price_prediction_EDA.ipynb) due to size and readibility.
***

### PreProcessing Steps

In [4]:
def drop_missing_cols_df (df):
    '''Drops the columns with 80% or hihgher missingness'''
    missing_cols = []  
    for col in df.columns:
        if df[col].isnull().sum()/df.shape[0] >= 0.8:
            missing_cols.append(col)
    dropped_df=df.drop(columns=missing_cols)
    return dropped_df, missing_cols  

In [5]:
def impute_missing_val_df (df):
    ''' Imputes the continious columns with mean and categorical columns (which has 80% missingness) with the most frequent value'''
    imputer_con = SimpleImputer(missing_values=np.nan, strategy='mean')
    imputer_cat = SimpleImputer(missing_values=np.nan, strategy='most_frequent')
    for col in df.columns:
        if df[col].isnull().sum() > 0:    
            if df[col].dtype.name == 'object':
                 df[col] = imputer_cat.fit_transform(df[col].values.reshape(-1,1))
            else:            
                 df[col] = imputer_con.fit_transform(df[col].values.reshape(-1,1))
    return df      

In [6]:
def convert_data_types_df (df):
    ''' converts the types of data columns into the appropriate type'''
    for col in df.columns: 
        if df[col].dtype.name == 'object':
             df[col]=df[col].astype('category')
        else:
             df[col]=df[col].astype('float')
    df['MSSubClass'] = df['MSSubClass'].astype('category')
    df['MoSold'] = df['MoSold'].astype('category')  
    return df            

In [7]:
def get_target_df(df, target):
    '''returns target dataframe'''
    return df[target]

In [8]:
def drop_columns_df (df, drop_cols):
    '''drop the specified columns from the dataframe'''
    df=df.drop (columns=drop_cols)
    return df

In [9]:
def list_of_cat_and_cont_vars (df):
    '''determine the categorical and continous variables'''
    categorical_cols = []
    numerical_cols = []
    for col in df.columns:
        if df[col].dtype.name == 'category':
            categorical_cols.append(col)
        else: 
            numerical_cols.append(col)
    numerical_cols.remove('SalePrice')       
    return categorical_cols, numerical_cols

In [10]:
def one_hot_encode_feature_df(df, cat_vars, num_vars):
    '''performs one-hot encoding on all categorical variables and combines result with continous variables'''
    cat_df = pd.get_dummies(df[cat_vars])
    num_df = df[num_vars].apply(pd.to_numeric)
    return pd.concat([cat_df, num_df], axis=1)#,ignore_index=False)

In [11]:
# drop the missing columns and keep track of columns to be dropped
dropped_train_df, missing_cols_train_df =  drop_missing_cols_df (df_train) 
dropped_test_df = drop_columns_df (df_test_feature, missing_cols_train_df) # drop the same columns from the test features to ensure consistency

In [12]:
#impute the missing values in a given row and col
imputed_train_df = impute_missing_val_df (dropped_train_df)
imputed_test_df = impute_missing_val_df (dropped_test_df)

In [13]:
# Convert the types to categorical and numeric columns to float
type_converted_train_df = convert_data_types_df (imputed_train_df)
type_converted_test_df = convert_data_types_df (imputed_test_df)

In [14]:
# Iterate and get the list of categorical and numerical features
categorical_vars, numerical_vars = list_of_cat_and_cont_vars(type_converted_train_df)
target_var = 'SalePrice'

In [15]:
#encode categorical data and get final feature dfs
feature_train_df = one_hot_encode_feature_df(type_converted_train_df, categorical_vars, numerical_vars)
feature_test_df  = one_hot_encode_feature_df(type_converted_test_df,  categorical_vars, numerical_vars)

In [16]:
#get target column from training and test dataframes
target_train_df = get_target_df(df_train, target_var)
target_test_df  = get_target_df(df_test_target, target_var)

In [17]:
feature_final_train, feature_final_test = feature_train_df.align(feature_test_df, join='inner', axis=1)  # inner join

### ---- Establish a baseline ----

In [25]:
# Create linear regression object
regr = LinearRegression()
regr.fit(feature_final_train, target_train_df)
y_pred = regr.predict(feature_final_test)
print ('\n MAE for OLS: %.2f' % mean_absolute_error(target_test_df, y_pred))
print ('\n RMSE for OLS: %.2f' % np.sqrt(mean_squared_error(target_test_df, y_pred)))


 MAE for OLS: 56469.50

 RMSE for OLS: 75136.47


### ---- Hypothesize solution ----

I believe the MAE can be improved by one of the following three popular tree-based ML methods. These methods have been shown to be succesful in competations like Kaggle. So let's try and compare their performance on housing dataset.

- Random Forest
- Xgboost 
- Lightgbm

## Part 3 - DEVELOP

In this part of the process, I'll look into creating features, tuning models, and training/validing models

### ---- Engineer features  ----

At this point, I'll decide against engineering and adding any feature and proceed to training and testing the models.

### ---- Create models ----

I'll create and tune the models that I brainstormed eariler

In [32]:
# Initializing Regressors
reg1 = RandomForestRegressor(random_state=1)
reg2 = XGBRegressor(random_state=1)
reg3 = LGBMRegressor(random_state=1)

In [53]:
# Setting up the parameter grids
param_grid1 = { 'n_estimators': [1000, 5000, 10000, 25000, 50000]}

param_grid2 = {
            'max_depth': range (2, 10, 1),
            'n_estimators': range(60, 220, 40),
            'learning_rate': [0.1, 0.01, 0.05]
        }

param_grid3 = {'num_leaves': (45, 60),
              'feature_fraction': (0.1, 0.9),
              'bagging_fraction': (0.8, 1),
              'max_depth': (9, 13),
              'min_split_gain': (0.001, 0.1),
              'min_child_weight': (30, 50)
              }

In [37]:
# Setting up multiple GridSearchCV objects, 1 for each algorithm
gridcvs = {}
inner_cv = KFold(n_splits=2, shuffle=True, random_state=1)

for pgrid, est, name in zip((param_grid1, param_grid2, param_grid3),
                            (reg1, reg2, reg3),
                            ('RForest', 'Xgboost', 'LightGBM')):
    gcv = GridSearchCV(estimator=est,
                       param_grid=pgrid,
                       n_jobs=-1,
                       cv=inner_cv,
                       verbose=0,
                       refit=True)
    gridcvs[name] = gcv

In [38]:
outer_cv = KFold(n_splits=5, shuffle=True, random_state=1)

for name, gs_est in sorted(gridcvs.items()):
    scores_dict = cross_validate(gs_est, 
                                 X=feature_final_train, 
                                 y=target_train_df, 
                                 cv=outer_cv,
                                 return_estimator=True,
                                 n_jobs=-1)

    print(50 * '-', '\n')
    print('Algorithm:', name)
    print('    Inner loop:')
    
    
    for i in range(scores_dict['test_score'].shape[0]):

        print('\n        Best Score (avg. of inner test folds) %.2f%%' % (scores_dict['estimator'][i].best_score_ * 100))
        print('        Best parameters:', scores_dict['estimator'][i].best_estimator_)
        print('        Score (on outer test fold) %.2f%%' % (scores_dict['test_score'][i]*100))

    print('\n%s | outer Score %.2f%% +/- %.2f' % 
          (name, scores_dict['test_score'].mean() * 100, 
           scores_dict['test_score'].std() * 100))

-------------------------------------------------- 

Algorithm: LightGBM
    Inner loop:

        Best Score (avg. of inner test folds) 83.18%
        Best parameters: LGBMRegressor(bagging_fraction=0.8, feature_fraction=0.9, max_depth=9,
              min_child_weight=30, min_split_gain=0.001, num_leaves=45,
              random_state=1)
        Score (on outer test fold) 88.05%

        Best Score (avg. of inner test folds) 83.85%
        Best parameters: LGBMRegressor(bagging_fraction=0.8, feature_fraction=0.9, max_depth=13,
              min_child_weight=30, min_split_gain=0.001, num_leaves=45,
              random_state=1)
        Score (on outer test fold) 84.59%

        Best Score (avg. of inner test folds) 85.26%
        Best parameters: LGBMRegressor(bagging_fraction=0.8, feature_fraction=0.9, max_depth=9,
              min_child_weight=30, min_split_gain=0.001, num_leaves=45,
              random_state=1)
        Score (on outer test fold) 88.01%

        Best Score (avg. of

### ---- 9 Test models ----

In [56]:
#do 2-fold cross validation on the best model and measure RMSE
gcv_model_select = GridSearchCV(estimator=reg1,
                                param_grid=param_grid1,
                               # scoring='r2',
                                n_jobs=-1,
                                cv= inner_cv,
                                verbose=1,
                                refit=True)

gcv_model_select.fit(feature_final_train, target_train_df)

Fitting 2 folds for each of 5 candidates, totalling 10 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   6 out of  10 | elapsed:  2.9min remaining:  2.0min
[Parallel(n_jobs=-1)]: Done  10 out of  10 | elapsed: 27.6min finished


GridSearchCV(cv=KFold(n_splits=2, random_state=1, shuffle=True),
             estimator=RandomForestRegressor(random_state=1), n_jobs=-1,
             param_grid={'n_estimators': [1000, 5000, 10000, 25000, 50000]},
             verbose=1)

### ---- 10 Select best model  ----

In [57]:
#select the model with the lowest error as your "prodcuction" model
best_model = gcv_model_select.best_estimator_


train_RMSE = np.sqrt(mean_squared_error(y_true=target_train_df, y_pred=best_model.predict(feature_final_train)))
test_RMSE  = np.sqrt(mean_squared_error(y_true=target_test_df,  y_pred=best_model.predict(feature_final_test)))

print('R2 score: %.2f%% (average over k-fold CV test folds)' %
      (100 * gcv_model_select.best_score_))
print('Best Parameters: %s' % gcv_model_select.best_params_)

print('Training RMSE: %.2f' % (train_RMSE))
print('Test RMSE: %.2f' % (test_RMSE))

R2 score: 85.73% (average over k-fold CV test folds)
Best Parameters: {'n_estimators': 10000}
Training RMSE: 10769.24
Test RMSE: 70017.55


## Part 4 - DEPLOY

### ---- 11 Automate pipeline ----

In [ ]:
#write script that trains model on entire training set, saves model to disk,
#and scores the "test" dataset

# Preprocessing for numerical data
numerical_transformer = SimpleImputer(strategy='constant')

# Preprocessing for categorical data
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])
# Preprocessing for numerical and categorical data
preprocessor = ColumnTransformer(
        transformers =[
            ('num', numerical_transformer, numerical_cols),
            ('cat', categorical_transformer, categorical_cols)
        ])

### ---- 12 Deploy solution ----

In [ ]:
#save your prediction to a csv file or optionally save them as a table in a SQL database
#additionally, you want to save a visualization and summary of your prediction and feature importances
#these visualizations and summaries will be extremely useful to business stakeholders

### ---- 13 Measure efficacy ----

We'll skip this step since we don't have the outcomes for the test data